In [8]:
# 1. Imports: sqlite3 = SQL-Datenbank-Tool (kostenlos, immer dabei)
# pandas = für to_sql() und read_sql_query()

# ALLGEMEINE INFO: 
# Wir haben im ersten Projekt eine CSV von Kaggle runtergeladen 
# wo wir die Datenanalyse per pandas gemacht haben. 
# Nun möchten wir dies über SQL machen und müssen dafür 
# die CSV erstmal zu einer Datenbank machen, 
# weil SQL über Datenbanken kommuniziert

import sqlite3 as sql # -> sql.connect() erstellt DB
import pandas as pd # -> df.to_sql() macht CSV zur DB


In [9]:
# 2. CSV-Laden: Superstore-Daten von GitHub (9.994 Zeilen, 21 Spalten)
# Warum GitHub-Link? -> Kaggle-Version hatte fehlende relevante/wichtige Spalten

url = "https://gist.githubusercontent.com/nnbphuong/38db511db14542f3ba9ef16e69d3814c/raw/Superstore.csv"
df = pd.read_csv(url) # Lädt direkt aus Internet (kein Download nötig!)

# Daten Prüfen
print(f" CSV geladen: {df.shape[0]:,} Zeilen, {df.shape[1]} Spalten")
print("Alle Spalten (vollständig):")
print(df.columns.tolist())
print("\n Erste 3 Zeilen:")
df.head(3)


 CSV geladen: 9,994 Zeilen, 21 Spalten
Alle Spalten (vollständig):
['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

 Erste 3 Zeilen:


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [10]:
# 3. DB erstellen
conn = sql.connect('superstore.db') # Öffne/erstelle Datei 'superstore.db'
df.to_sql('superstore', conn, if_exists='replace', index=False) # Wenn Datei nicht existiert -> Neue leere DB wird erstellt (0 Bytes), wenn Datei existiert -> alte tabelle löschen + neu erstellen bzw überschreiben, index=false -> pandas index weglassen
print("SQL-Tabelle erstellt!")

SQL-Tabelle erstellt!


In [11]:
# ABFRAGE 1: "Welche Kategorien sind umsatzstark/profitabel?"
# Pandas-Äquivalent: df.groupby('Category')[['Sales','Profit']].agg(['mean','sum'])
query1 = """
SELECT 
Category, 
COUNT(*) as Anzahl_Orders,
ROUND(AVG(Sales), 0) as Durchschnitt_Sales,
ROUND(SUM(Sales), 0) as Gesamt_Sales,
ROUND(AVG(Profit), 0) as Durchschnitt_Profit,
ROUND(SUM(Profit), 0) as Gesamt_Profit
FROM superstore
GROUP BY Category
ORDER BY Durchschnitt_Sales DESC
LIMIT 5
"""
ergebnis_kategorien = pd.read_sql_query(query1, conn)
print("KATEGORIEN‑RANKING:")
print(ergebnis_kategorien)

KATEGORIEN‑RANKING:
          Category  Anzahl_Orders  Durchschnitt_Sales  Gesamt_Sales  \
0       Technology           1847               453.0      836154.0   
1        Furniture           2121               350.0      742000.0   
2  Office Supplies           6026               119.0      719047.0   

   Durchschnitt_Profit  Gesamt_Profit  
0                 79.0       145455.0  
1                  9.0        18451.0  
2                 20.0       122491.0  


In [12]:
query2 = """
SELECT 
    "Customer ID" as Kunde,          -- Welcher Kunde?
    "Order ID" as orderNr,             -- Welche Bestellung?
    Sales,                -- Umsatz
    Profit,               -- Gewinn (in €)
    Category,             -- Produktgruppe
    "Product Name" as Produkt          -- Genauer Artikel
FROM superstore
WHERE Profit > 2000        -- NUR High‑Profit Orders!
ORDER BY Profit DESC       -- Beste zuerst
LIMIT 10                   -- Top 10
"""

ergebnis2 = pd.read_sql_query(query2, conn)
print("TOP 10 GEWINN‑BESTELLUNGEN:")
print(ergebnis2)

TOP 10 GEWINN‑BESTELLUNGEN:
      Kunde         orderNr      Sales     Profit         Category  \
0  TC-20980  CA-2017-118689  17499.950  8399.9760       Technology   
1  RB-19360  CA-2018-140151  13999.960  6719.9808       Technology   
2  HL-15040  CA-2018-166709  10499.970  5039.9856       Technology   
3  AB-10105  CA-2017-117121   9892.740  4946.3700  Office Supplies   
4  SC-20095  CA-2015-116904   9449.950  4630.4755  Office Supplies   
5  TA-21385  CA-2018-127180  11199.968  3919.9888       Technology   
6  CM-12385  CA-2016-145352   6354.950  3177.4750  Office Supplies   
7  SE-20110  CA-2017-158841   8749.950  2799.9840       Technology   
8  DR-12940  US-2017-140158   5399.910  2591.9568       Technology   
9  AR-10540  CA-2018-138289   5443.960  2504.2216  Office Supplies   

                                             Produkt  
0              Canon imageCLASS 2200 Advanced Copier  
1              Canon imageCLASS 2200 Advanced Copier  
2              Canon imageCLASS 2200

In [13]:
# ABFRAGE 3: "Welche Regionen sind unsere Geldmaschinen?"
# Business: Marktanteile pro Region
query3 = """
/* Regionen‑Vergleich: Umsatz + Gewinn + Order‑Anzahl */
SELECT 
    Region,
    COUNT(*) as Anzahl_Orders,           -- Wie aktiv ist Region?
    ROUND(SUM(Sales), 0) as Total_Sales, -- Gesamtumsatz
    ROUND(SUM(Profit), 0) as Total_Profit, -- Gesamtgewinn
    ROUND(AVG(Profit), 0) as Profit_pro_Order -- Gewinn/Order
FROM superstore
GROUP BY Region
ORDER BY Total_Sales DESC
"""

ergebnis3 = pd.read_sql_query(query3, conn)
print("REGIONEN‑RANKING:")
ergebnis3

REGIONEN‑RANKING:


,Region,Anzahl_Orders,Total_Sales,Total_Profit,Profit_pro_Order
0,West,3203,725458.0,108418.0,34.0
1,East,2848,678781.0,91523.0,32.0
2,Central,2323,501240.0,39706.0,17.0
3,South,1620,391722.0,46749.0,29.0


In [14]:
# ABFRAGE 4: Monatliche Entwicklung (mit DATE‑Funktion)
query4 = """
/* Umsatz pro Monat (Zeitreihen‑Analyse!) */
SELECT 
    strftime('%Y-%m', "Order Date") as Monat_Jahr,  -- Monat extrahieren
    COUNT(*) as Orders,
    ROUND(SUM(Sales), 0) as Monats_Umsatz
FROM superstore
GROUP BY Monat_Jahr
ORDER BY Monat_Jahr
"""

ergebnis4 = pd.read_sql_query(query4, conn)
print("UMSATZ ENTWICKLUNG:")
ergebnis4.head(12)  # Erstes Jahr

UMSATZ ENTWICKLUNG:


,Monat_Jahr,Orders,Monats_Umsatz
0,2015-01,79,14237.0
1,2015-02,46,4520.0
2,2015-03,157,55691.0
3,2015-04,135,28295.0
4,2015-05,122,23648.0
5,2015-06,135,34595.0
6,2015-07,143,33946.0
7,2015-08,153,27909.0
8,2015-09,268,81777.0
9,2015-10,159,31453.0


In [ ]:
# ABFRAGE 5: "Welche Kunden machen VERLUST?" (HAVING)
# Business: Kunden‑Portfolio optimieren
query5 = """
SELECT 
    "Customer ID" as Kunde,
    "Customer Name" as Name,
    COUNT(*) as Anzahl_Orders,
    ROUND(SUM(Sales), 0) as Total_Sales,
    ROUND(SUM(Profit), 0) as Total_Profit  -- Negativ = Problem!
FROM superstore
GROUP BY "Customer ID", "Customer Name"
HAVING Total_Profit < -4000      -- Top Verlust‑Maker
ORDER BY Total_Profit ASC        -- Schlimmste zuerst
LIMIT 5
"""

ergebnis5 = pd.read_sql_query(query5, conn)
print("TOP 5 VERLUST‑KUNDEN:")
ergebnis5

TOP 5 VERLUST‑KUNDEN (Action Item!):


,Kunde,Name,Anzahl_Orders,Total_Sales,Total_Profit
0,CS-12505,Cindy Stewart,9,5690.0,-6626.0
1,GT-14635,Grant Thornton,6,9351.0,-4109.0


In [16]:
# NOTEBOOK‑ENDE: DB schließen + Summary
# =============================================================================

# DB SICHER SPEICHERN
conn.close()
print("Datenbank 'superstore.db' gespeichert!")

print("""
 **SUPERSTORE SQL ANALYSIS – ABGESCHLOSSEN!**

5 Business‑Queries geliefert:
1. Kategorien‑Ranking → Technology gewinnt
2. Top Orders → Canon Copier = Star 
3. Regionen → West dominiert (725k €)
4. Monats‑Trends → Sept Peak
5. Verlust‑Kunden → Optimierungspotenzial
""")

Datenbank 'superstore.db' gespeichert!

 **SUPERSTORE SQL ANALYSIS – ABGESCHLOSSEN!**

5 Business‑Queries geliefert:
1. Kategorien‑Ranking → Technology gewinnt
2. Top Orders → Canon Copier = Star 
3. Regionen → West dominiert (725k €)
4. Monats‑Trends → Sept Peak
5. Verlust‑Kunden → Optimierungspotenzial

